In [1]:
import cx_Oracle 
import warnings
warnings.simplefilter("ignore")
import mysql.connector
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill, colors, Font, Color, Alignment, Border, Side
from datetime import datetime, timedelta, date
import math

# Conexão ORACLE Produção

In [2]:
def ConectaOracle1():
    global conn
    global cursorORC
    global DFProjetos
    
    sconexao = "bismarckhegermann/mapa1234@masrv-scan.agricultura.gov.br/prodmapa.agricultura.gov.br"
    conn = cx_Oracle.connect(sconexao) 

    print("Conectado ao servidor ORACLE - PRODUÇÃO")
    cursorORC = conn.cursor()
    return

# PRODUÇÃO - ORACLE - LOAD PROJETOS

In [3]:
def ConectaOracle():
    global conn
    global cursorORC
    global DFProjetos
    
    sconexao = "bismarckhegermann/mapa1234@masrv-scan.agricultura.gov.br/prodmapa.agricultura.gov.br"
    conn = cx_Oracle.connect(sconexao) 

    print("Conectado ao servidor ORACLE - PRODUÇÃO")
    cursorORC = conn.cursor()
    print("Selecionando os Projetos")
    
    SqlProj =   "SELECT \
                Ativid.id_projeto,\
                Ativid.id_atividade,\
                realiz.id_ticket,\
                to_char(realiz.nr_ticket) NrTicket,\
                ativid.ds_atividade,\
                ativid.ds_servico,\
                Proj.nm_projeto,\
                to_char(Proj.dt_cadastro,'DD/MM/RRRR'),\
                coord.sg_coordenacao,\
                ativid.nm_fila,\
                status.ds_situacao \
            FROM \
                governanca_cgti_mapa.s_projeto_planejamento_ativid Ativid \
            left JOIN \
                governanca_cgti_mapa.s_projeto_planejamento_realiz Realiz \
            ON \
              Ativid.id_atividade = realiz.id_atividade_planejada \
            AND \
              ativid.id_planejamento = realiz.id_planejamento \
            AND \
                ativid.id_fase = realiz.id_fase \
            AND \
                Ativid.id_projeto = Realiz.id_projeto \
            INNER JOIN \
                governanca_cgti_mapa.s_projeto Proj \
            ON \
                Ativid.id_projeto = Proj.id_projeto \
            INNER JOIN \
                governanca_cgti_mapa.s_projeto_coord_gestora Coord \
            ON \
                Proj.id_coordenacao_gestora = coord.id_coordenacao \
            INNER JOIN \
                governanca_cgti_mapa.s_projeto_situacao status \
            ON \
                Proj.id_situacao = status.id_situacao \
            WHERE \
                    status.ds_situacao != 'Cancelado' and \
                    status.ds_situacao != 'Suspenso' and \
                    status.ds_situacao != 'Concluído' and \
                    status.ds_situacao != 'Em encerramento' \
            UNION ALL \
                SELECT \
                    realiz.id_projeto,\
                    realiz.id_planejamento, \
                    realiz.id_ticket,\
                    to_char(realiz.nr_ticket),\
                    realiz.ds_atividade,\
                    TO_CHAR(NULL),\
                    Proj.nm_projeto,\
                    to_char(Proj.dt_cadastro, 'DD/MM/RRRR'),\
                    coord.sg_coordenacao,\
                    TO_CHAR('NÃO PLANEJADA'),\
                    status.ds_situacao \
                FROM \
                    governanca_cgti_mapa.s_projeto_planejamento_realiz realiz \
                INNER JOIN \
                    governanca_cgti_mapa.s_projeto Proj \
                ON \
                    realiz.id_projeto = Proj.id_projeto \
                INNER JOIN \
                    governanca_cgti_mapa.s_projeto_coord_gestora Coord \
                ON \
                    Proj.id_coordenacao_gestora = coord.id_coordenacao \
                INNER JOIN \
                    governanca_cgti_mapa.s_projeto_situacao status \
                ON \
                    Proj.id_situacao = status.id_situacao \
                where \
                    realiz.id_planejamento is null \
                and \
                    realiz.id_atividade_planejada is null \
                and \
                    status.ds_situacao != 'Cancelado' and \
                    status.ds_situacao != 'Suspenso' and \
                    status.ds_situacao != 'Concluído' and \
                    status.ds_situacao != 'Em encerramento'\
                ORDER BY 1, 2"
        
    cursorORC.execute(SqlProj)
    
    row = cursorORC.fetchall() 
    DFProjetos =  pd.DataFrame(row, columns = [ "id_projeto",\
                                                "id_atividade", \
                                                "id_ticket", \
                                                "NrTicket", \
                                                "ds_atividade", \
                                                "ds_servico", \
                                                "nm_projeto", \
                                                "dt_cadastro", \
                                                "sg_coordenacao", \
                                                "Desc_Torre", \
                                                "Status"])
    
    DFProjetos = DFProjetos.sort_values(by = ["id_projeto", "id_atividade"])
    
    print("Fim da Selecão de Projetos")

#ConectaOracle()

# CONECTA MYSQL - OTRS - PRODUÇÃO

In [4]:
def ConectaMySql():
    global con
    global cursorMySql
    
    con = mysql.connector.connect(host='192.168.2.132', database='otrs', port=3306, user='painel_otrs_apl',password='painel_otrs_apl_1234', buffered=True)
    
    if con.is_connected():
        db_info = con.get_server_info()
        print("Conectado ao servidor MySQL PRODUÇÃO versão ",db_info)
        cursorMySql = con.cursor() 
    else:
        print("Não Conectado ao MYSQL")
#ConectaMySql()

# BUSCA OBSERVAÇÕES - OTRS

In [5]:
def BuscaOBS(Ticket): 
    SQL  = "select \
                T.id IDTicket, \
                T.tn NrTicket, \
                A.id IDArtigo, \
                T.title, \
                S.name StatusTicket, \
                DATE_FORMAT(T.create_time, '%d/%m/%Y %H:%i'), \
                C.a_body Descrição, \
                Serv.name, \
                C.a_from, \
                DATE_FORMAT(C.create_time, '%d/%m/%Y %H:%i') \
            from \
                ticket T \
            INNER JOIN \
                article A \
            ON \
                T.id = A.ticket_id \
            INNER JOIN \
                article_data_mime C \
            ON \
                A.id = C.article_id \
            INNER JOIN \
                ticket_state S \
            ON \
                T.ticket_state_id = S.id \
            INNER JOIN \
                service Serv \
            ON \
                T.service_id = Serv.id \
            where T.tn = " + str(Ticket) + " \
            order by C.create_time"
        
    cursorMySql.execute(SQL)
    reg = cursorMySql.fetchone()
    RegOTRS = []
    while reg:
        RegOTRS.append(reg)
        reg = cursorMySql.fetchone()
        
    return(RegOTRS)

#BuscaOBS("2020121706009416")

# Funções de Configuração da Planilha EXCEL

In [6]:
def ConfCell(Linha):
    pasta.cell(row = Linha, column = 1).font = big_red_text
    pasta.cell(row = Linha, column = 1).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 1).border  = square_border
    pasta.cell(row = Linha, column = 1).fill = cinza_background        
    
    pasta.cell(row = Linha, column = 2).font = big_red_text
    pasta.cell(row = Linha, column = 2).border  = square_border
    pasta.cell(row = Linha, column = 2).fill = cinza_background 
    pasta.cell(row = Linha, column = 2).alignment = center_aligned_text

    pasta.cell(row = Linha, column = 3).font = big_red_text
    pasta.cell(row = Linha, column = 3).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 3).border  = square_border
    pasta.cell(row = Linha, column = 3).fill = cinza_background        

    pasta.cell(row = Linha, column = 4).font = big_red_text
    pasta.cell(row = Linha, column = 4).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 4).border  = square_border
    pasta.cell(row = Linha, column = 4).fill = cinza_background        

    pasta.cell(row = Linha, column = 5).font = big_red_text
    pasta.cell(row = Linha, column = 5).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 5).border  = square_border
    pasta.cell(row = Linha, column = 5).fill = cinza_background  

    pasta.cell(row = Linha, column = 6).font = big_red_text
    pasta.cell(row = Linha, column = 6).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 6).border  = square_border
    pasta.cell(row = Linha, column = 6).fill = cinza_background  
    
    pasta.cell(row = Linha, column = 7).font = big_red_text
    pasta.cell(row = Linha, column = 7).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 7).border  = square_border
    pasta.cell(row = Linha, column = 7).fill = cinza_background  
    pasta.row_dimensions[Linha].height = 20
    return

def ConfGrade(Linha):
    pasta.cell(row = Linha, column = 1).border  = square_border
    pasta.cell(row = Linha, column = 2).border  = square_border
    pasta.cell(row = Linha, column = 3).border  = square_border
    pasta.cell(row = Linha, column = 4).border  = square_border
    pasta.cell(row = Linha, column = 5).border  = square_border
    pasta.cell(row = Linha, column = 6).border  = square_border
    pasta.cell(row = Linha, column = 7).border  = square_border
    pasta.cell(row = Linha, column = 1).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 2).alignment = center_aligned_text_Vert
    pasta.cell(row = Linha, column = 3).alignment = center_aligned_text_Vert
    pasta.cell(row = Linha, column = 4).alignment = center_aligned_text_Vert
    pasta.cell(row = Linha, column = 5).alignment = center_aligned_text
    pasta.cell(row = Linha, column = 6).alignment = center_aligned_text
    pasta.row_dimensions[Linha].height = 20
    return

def QuebraLinha(linha):
    default_height = 12.5
    multiples_of_font_size2 = [default_height]
    multiples_of_font_size3 = [default_height]
    multiples_of_font_size4 = [default_height]
    wrap_text = True
    vertical = "center"
    mul = 0
    
    for v in str(pasta.cell(row = Linha, column = 2).value).split('\n'):
        mul += math.ceil(len(v) / 51) * pasta.cell(row = Linha, column = 2).font.size

        if mul > 0:
            multiples_of_font_size2.append(mul)

    for v in str(pasta.cell(row = Linha, column = 3).value).split('\n'):
        mul += math.ceil(len(v) / 51) * pasta.cell(row = Linha, column = 3).font.size

        if mul > 0:
            multiples_of_font_size3.append(mul)

    for v in str(pasta.cell(row = Linha, column = 4).value).split('\n'):
        mul += math.ceil(len(v) / 51) * pasta.cell(row = Linha, column = 4).font.size

        if mul > 0:
            multiples_of_font_size4.append(mul)

    pasta.cell(row = Linha, column = 2).alignment = Alignment(wrap_text=wrap_text, vertical=vertical)
    pasta.cell(row = Linha, column = 3).alignment = Alignment(wrap_text=wrap_text, vertical=vertical)
    pasta.cell(row = Linha, column = 4).alignment = Alignment(wrap_text=wrap_text, vertical=vertical)

    original_height = pasta.row_dimensions[Linha].height
    
    if original_height is None:
        original_height = default_height
        
    VetorTotal = multiples_of_font_size2 + multiples_of_font_size3 + multiples_of_font_size4

    new_height = max(VetorTotal)
    
    if original_height < new_height:
        pasta.row_dimensions[Linha].height = new_height
    return

def CalcData(Data, Linha, Status):
    Dif = 999
    if Status == "Cancelado" or Status == "Em encerramento" or Status == "Encerrado" or Status == "Resolvido" or Data == "999":
        Sinal = ["x" if Status == "Cancelado" else \
                 "@" if Status == "Em encerramento" else \
                 "r" if Data == "999" else \
                 "a"] 
        dif = [SinalStatus]
    else:    
        if Data != "999":
            d2 = datetime.strptime(Data, '%d/%m/%Y %H:%M')
            d1 = datetime.strptime(format(now, '%Y-%m-%d'), '%Y-%m-%d')
            Dif = abs(d1-d2).days
        if Status == "Ocor":
            Sinal = ["i"]
            dif = [SinalStatus]
        else:    
            dif = [SinalVerde if Dif >= 0 and Dif <= 3 else SinalRosa if Dif >= 4 and Dif <= 10 else SinalAmarelo if Dif > 10 and Dif <= 15 else SinalVermelho]    
            Sinal = ["n"]
        
    pasta.cell(row = Linha, column = 7).value = Sinal[0]
    pasta.cell(row = Linha, column = 7).font = dif[0]
    pasta.cell(row = Linha, column = 7).alignment = Alignment(horizontal="center", vertical="center")
    return

# GRAVA PROJETOS NO EXCEL 

In [7]:
IDProj = 0
reg = 0
Linha = 1
LinhaTitulo = 0
ConectaMySql()
ConectaOracle()

workbook = Workbook()
pasta = workbook.active
now = datetime.now()
nome_xlsx = "StatusOTRS" + str(now.day) + str(now.month) + str(now.year) + "_" + str(now.hour) + str(now.minute) + ".xlsx"
            
pasta.delete_rows(2, pasta.max_row)
pasta.column_dimensions["A"].width = 20 
pasta.column_dimensions["B"].width = 40
pasta.column_dimensions["C"].width = 40
pasta.column_dimensions["D"].width = 36 
pasta.column_dimensions["E"].width = 22
pasta.column_dimensions["F"].width = 19
pasta.column_dimensions["G"].width = 12 

SinalStatus = Font(name = "Webdings", bold=True, color="FF7CFC00", size=28)

SinalVerde = Font(name = "Webdings", bold=True, color="FF7CFC00", size=16)
SinalAmarelo = Font(name = "Webdings", bold=True, color="FFFFFF00", size=16)
SinalVermelho = Font(name = "Webdings", bold=True, color="FFFF0000", size=16)
SinalPreto = Font(name = "Webdings", bold=True, color="FF000000", size=16)
SinalRosa = Font(name = "Webdings", bold=True, color="FFFFE4B5", size=16)

big_red_text = Font(bold=True, color=colors.BLACK, size=12)
center_aligned_text = Alignment(horizontal="center", vertical="center")
double_border_side = Side(border_style="thin")
square_border = Border(top=double_border_side, \
                       right=double_border_side, \
                       bottom=double_border_side, \
                       left=double_border_side)

center_aligned_text_Vert = Alignment(vertical="center")

cinza_background = PatternFill(start_color='CCCCCC', end_color='CCCCCC', fill_type="solid")

pasta.cell(row = Linha, column = 1).value = "ID DO PROJETO"
pasta.cell(row = Linha, column = 2).value = "PROJETO"
pasta.cell(row = Linha, column = 4).value = "DATA DA ABERTURA"
pasta.cell(row = Linha, column = 5).value = "STATUS PROJ."
pasta.cell(row = Linha, column = 6).value = "COORDENAÇÃO"
pasta.cell(row = Linha, column = 7).value = "STATUS"
pasta.merge_cells("B" + str(Linha) + ":c" + str(Linha), 'Merged Range')
ConfGrade(Linha)
ConfCell(Linha)

while reg < len(DFProjetos):
    
    status = ""
      
    if DFProjetos["id_projeto"][reg] != IDProj:
        Linha += 1
        FlagTitulo = 1

        pasta.cell(row = Linha, column = 1).value = str(DFProjetos["id_projeto"][reg])
        pasta.cell(row = Linha, column = 2).value = str(DFProjetos["nm_projeto"][reg])
        pasta.cell(row = Linha, column = 4).value = str(DFProjetos["dt_cadastro"][reg])
        pasta.cell(row = Linha, column = 5).value = str(DFProjetos["Status"][reg])
        pasta.cell(row = Linha, column = 6).value = str(DFProjetos["sg_coordenacao"][reg])

        pasta.merge_cells("B" + str(Linha) + ":c" + str(Linha), 'Merged Range')
        LinhaTitulo = Linha       

        ConfCell(Linha)

    else:
        Linha += 1
        pasta.cell(row = Linha, column = 1).value = str(DFProjetos["NrTicket"][reg])
        pasta.cell(row = Linha, column = 2).value = str(DFProjetos["ds_atividade"][reg])
        pasta.cell(row = Linha, column = 3).value = str(DFProjetos["Desc_Torre"][reg])
        CalcData("999", Linha, status) 

        ConfGrade(Linha)
        QuebraLinha(Linha)

        if pd.isnull(DFProjetos["NrTicket"][reg]) != True: 
            regOTRS = BuscaOBS(DFProjetos["NrTicket"][reg])
            ContOcor = 0

            for CamposOtrs in regOTRS:

                if CamposOtrs[4] != "Encerrado" and \
                   CamposOtrs[4] != "Resolvido" and \
                   CamposOtrs[4] != "Cancelado":

                    if ContOcor > 0:
                        pasta.cell(row = Linha, column = 1).value = "Ocor.:" + str(ContOcor) 
                        CalcData(CamposOtrs[9], Linha, "Ocor")
                    else:
                        pasta.cell(row = Linha, column = 1).font = big_red_text
                        pasta.cell(row = Linha, column = 6).value = CamposOtrs[4]  
                        CalcData(CamposOtrs[9], Linha, CamposOtrs[4]) 

                    pasta.cell(row = Linha, column = 2).value = CamposOtrs[6]
                    pasta.cell(row = Linha, column = 3).value = CamposOtrs[7]
                    pasta.cell(row = Linha, column = 4).value = CamposOtrs[8]
                    pasta.cell(row = Linha, column = 5).value = CamposOtrs[9]

                    ConfGrade(Linha)                        
                    QuebraLinha(Linha)       
                    ContOcor += 1
                    Linha += 1
                    FlagTitulo = 0
                    
                else:
                    pasta.cell(row = Linha, column = 4).value = CamposOtrs[8]
                    pasta.cell(row = Linha, column = 5).value = CamposOtrs[9]
                    pasta.cell(row = Linha, column = 6).value = CamposOtrs[4]  
                    CalcData(CamposOtrs[9], Linha, CamposOtrs[4])

                    ConfGrade(Linha)                        
                    QuebraLinha(Linha) 
                    break

            if ContOcor > 0:
                Linha -= 1

            print("Atualizado o TICKET", DFProjetos["NrTicket"][reg])
    
    if FlagTitulo == 0:
        pasta.cell(row = LinhaTitulo, column = 7).value = "Com Titulo"  
        pasta.cell(row = LinhaTitulo, column = 7).font = Font(color="CCCCCC")

    IDProj = DFProjetos["id_projeto"][reg]
    reg += 1    

workbook.save(filename = nome_xlsx)
workbook.close()

if con.is_connected():
    cursorMySql.close()
    con.close()
    print("Conexão ao MySQL foi encerrada")
    cursorORC.close()
    conn.close()
    print("Conexão ao ORACLE foi encerrada")

print("FIM DE GERAÇÃO DA PLANILHA DE STATUS")

Conectado ao servidor MySQL PRODUÇÃO versão  5.5.64-MariaDB
Conectado ao servidor ORACLE - PRODUÇÃO
Selecionando os Projetos
Fim da Selecão de Projetos
Atualizado o TICKET 2020081706001361
Atualizado o TICKET 2020081706001342
Atualizado o TICKET 2020081706001315
Atualizado o TICKET 2020073106003913
Atualizado o TICKET 2020081306001055
Atualizado o TICKET 2020081006000972
Atualizado o TICKET 2020081006000936
Atualizado o TICKET 2020081106000292
Atualizado o TICKET 2020070706000695
Atualizado o TICKET 2020070706002148
Atualizado o TICKET 2020070706002157
Atualizado o TICKET 2020092806009376
Atualizado o TICKET 2020073106003511
Atualizado o TICKET 2020060406007221
Atualizado o TICKET 2020062606002177
Atualizado o TICKET 2020060406008104
Atualizado o TICKET 2020060506002117
Atualizado o TICKET 2020070706000515
Atualizado o TICKET 2020062306005947
Atualizado o TICKET 2020052606005578
Atualizado o TICKET 2020060206006977
Atualizado o TICKET 2020060206007038
Atualizado o TICKET 20200527060029

Atualizado o TICKET 2020110306002156
Atualizado o TICKET 2020092906002479
Atualizado o TICKET 2020080306007103
Atualizado o TICKET 2020080306005132
Atualizado o TICKET 2020081906005657
Atualizado o TICKET 2020082006004664
Atualizado o TICKET 2020061606001357
Atualizado o TICKET 2020061606001419
Atualizado o TICKET 2020062406002261
Atualizado o TICKET 2020052806005289
Atualizado o TICKET 2020052906000853
Atualizado o TICKET 2020060206001436
Atualizado o TICKET 2020081206011573
Atualizado o TICKET 2020110406009979
Atualizado o TICKET 2020092906005387
Atualizado o TICKET 2020080406003365
Atualizado o TICKET 2020080406002893
Atualizado o TICKET 2020081906009662
Atualizado o TICKET 2020061606001428
Atualizado o TICKET 2020072206005894
Atualizado o TICKET 2020062406002304
Atualizado o TICKET 2020052806006582
Atualizado o TICKET 2020052906001049
Atualizado o TICKET 2020060206004513
Atualizado o TICKET 2020081206011582
Atualizado o TICKET 2020110506001304
Atualizado o TICKET 2020080406002919
A